### PCA ###

Conjunto de datos de los arrestos según delito cometido (asesinato, violación, asalto), ademas contamos con el tamaño de la población humana, queremos saber si podemos reducir la cantidad de caracteristicas manteniendo la varianza explicada

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

# Advertencias 
import warnings
warnings.filterwarnings('ignore')

importamos los Datos 

In [3]:
USarrests = sm.datasets.get_rdataset('USArrests', 'datasets')
data = USarrests.data
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Alabama to Wyoming
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Murder    50 non-null     float64
 1   Assault   50 non-null     int64  
 2   UrbanPop  50 non-null     int64  
 3   Rape      50 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 2.0+ KB


In [7]:
data


,Murder,Assault,UrbanPop,Rape
rownames,,,,
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6
Colorado,7.9,204,78,38.7
Connecticut,3.3,110,77,11.1
Delaware,5.9,238,72,15.8
Florida,15.4,335,80,31.9


In [15]:
data.mean(axis=0)

Murder        7.788
Assault     170.760
UrbanPop     65.540
Rape         21.232
dtype: float64

In [11]:
data.std(axis=0) 

Murder       4.355510
Assault     83.337661
UrbanPop    14.474763
Rape         9.366385
dtype: float64

In [14]:
data.var(axis=0) 

Murder        18.970465
Assault     6945.165714
UrbanPop     209.518776
Rape          87.729159
dtype: float64

Rango: rango = mean -/+ std
Rango de Rape = 21 +/- 9 = 11 -> 30

### PCA: ###
Reduce cantidad de  dimenciones usando componentes ortognonales entre si, maximisando la variable explicada

El rango determina un peso entre mayor el considera que es más importante, entonces a las variables cuantitateivas se les va a generar un proceso de escalamiento
cada variable se escala para evitar los sesgos por rango, usaremos estandarización, toda variable se puede escribir como normal(0,1)

estandarización: Xn = X- mean / (std)
Rango: -4 -> 4

Reducción: X-min(x) / Max(x) - Min(x)
Rango: 0 -> 1

CUALITATIVAS 
Se codifican para que el modelo las entienda 

Ordinal: Le da un orden alfabetico a las variables, según el orden alfabetico regresa 1,2,3... 

Matricial: Crea columnas apartir de las clases o varibles de la clase, pone un un en la que corresponde, 1-0-0, 0-1-0, 0-0-1

Si hay 20 o menos se usa matricial, si hay más se usa Ordinal

las ordinales muchas veces toca reducirlas o estandarizarlas 



In [16]:
#pipeline para crear y estandarizar el modelo #\
pca_pipeline = make_pipeline(MinMaxScaler(), PCA())
pca_pipeline.fit(data)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('pca', PCA())])

In [19]:
pca_model = pca_pipeline.named_steps['pca']
pd.DataFrame(data= pca_model.components_, columns= data.columns, index= ['PC1','PC2', 'PC3', 'PC4'])

,Murder,Assault,UrbanPop,Rape
PC1,0.547500,0.645931,0.229559,0.479916
PC2,-0.387958,-0.152124,0.880444,0.226196
PC3,-0.268188,-0.264693,-0.380839,0.844378
PC4,-0.691236,0.699692,-0.164568,-0.074435


In [20]:
print("Varianza explicada por componente:")
print(pca_model.explained_variance_ratio_)

Varianza explicada por componente:
[0.64272873 0.22804606 0.08097894 0.04824627]


In [21]:
print("Varianza acumulada explicada:")
#Se usaran los datos que acumulen el 80% de la varianza
variance_acum = pca_model.explained_variance_ratio_.cumsum()
print(variance_acum)

Varianza acumulada explicada:
[0.64272873 0.87077478 0.95175373 1.        ]


In [22]:
#Ya que los primeros dos superan el 80% de la varianza,
# se puede reducir a 2 componentes principales
print("Reduciendo a 2 componentes principales")
#pipeline para crear y estandarizar el modelo con 2 componentes principales
pca_pipeline = make_pipeline(MinMaxScaler(), PCA(2))
pca_pipeline.fit(data)
pca_final_model = pca_pipeline.named_steps['pca']
pd.DataFrame(data= pca_final_model.components_, columns= data.columns, index= ['PC1','PC2'])

Reduciendo a 2 componentes principales


,Murder,Assault,UrbanPop,Rape
PC1,0.547500,0.645931,0.229559,0.479916
PC2,-0.387958,-0.152124,0.880444,0.226196
